In [ ]:
# Cell 1 — Imports & backend
import os, random, numpy as np
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split

# Use TensorFlow as Keras backend (Keras 3)
try:
    keras.config.set_backend("tensorflow")
except Exception:
    pass

print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)

# Optional: reproducibility
SEED = 123
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)


In [ ]:
# Cell 2 — Data extraction (CIFAR-10), scaling, and stratified train/val split
from tensorflow import keras as tfkeras  # for datasets

# Load CIFAR-10 (50k train, 10k test)
(x_train, y_train), (test_x, test_y) = tfkeras.datasets.cifar10.load_data()
y_train = y_train.squeeze().astype(np.int64)
test_y  = test_y.squeeze().astype(np.int64)

# Scale to [0,1]
x_train = x_train.astype("float32") / 255.0
test_x  = test_x.astype("float32")  / 255.0

# Stratified validation split (10% of the 50k training set → 5k)
x_train_real, x_val, y_train_real, y_val = train_test_split(
    x_train, y_train, test_size=0.1, stratify=y_train, random_state=SEED
)

print("Train:", x_train_real.shape, y_train_real.shape)
print("Val:  ", x_val.shape,          y_val.shape)
print("Test: ", test_x.shape,         test_y.shape)


In [ ]:
# Cell 3 — Model definition and training (original toy CNN)
import keras
from keras import layers, callbacks, optimizers

num_classes = 10
input_shape = (32, 32, 3)

# --- Data augmentation (kept modest) ---
data_aug = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomTranslation(0.05, 0.05),
    # (No RandomZoom on 32x32 to avoid too much info loss)
], name="data_aug")

# --- Small but solid CNN ---
inputs = keras.Input(shape=input_shape)
x = data_aug(inputs)

# Block 1
x = layers.Conv2D(32, 3, padding="same", use_bias=False)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.Conv2D(32, 3, padding="same", use_bias=False)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.MaxPooling2D()(x)
x = layers.Dropout(0.25)(x)

# Block 2
x = layers.Conv2D(64, 3, padding="same", use_bias=False)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.Conv2D(64, 3, padding="same", use_bias=False)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.MaxPooling2D()(x)
x = layers.Dropout(0.25)(x)

# Head
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs, name="cifar10_cnn_toy")

model.compile(
    optimizer=optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# --- Useful callbacks ---
cbs = [
    callbacks.ReduceLROnPlateau(
        monitor="val_accuracy", factor=0.5, patience=2, verbose=1, min_lr=3e-5
    ),
    callbacks.EarlyStopping(
        monitor="val_accuracy", patience=4, restore_best_weights=True
    ),
]

# --- Train ---
history = model.fit(
    x_train_real, y_train_real,
    validation_data=(x_val, y_val),
    epochs=30,                 # EarlyStopping will cap this
    batch_size=1024,           # On Colab T4 you can try bigger; on CPU lower if needed
    verbose=1,
    callbacks=cbs
)

# --- Evaluate on the held-out test set ---
test_loss, test_acc = model.evaluate(test_x, test_y, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")
